In [1]:
import pandas as pd
from src.load_data import loader
import pingouin as pg
import numpy as np

/Users/yifanzhang/miniconda3/envs/legal/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
data_all = pd.read_table('data/All.tsv', header=0, delimiter='\t', encoding='utf-16', encoding_errors='replace', keep_default_na=False)
print(list(data_all.columns))

['StartDate', 'EndDate', 'Status', 'Progress', 'Duration', 'Finished', 'RecordedDate', 'ResponseId', 'DistributionChannel', 'UserLanguage', 'Q_RecaptchaScore', 'Q1', 'Q2', 'Q3_1', 'Q1_2', 'Q1_3', 'Q42', 'Q1.2', 'Q1.3', 'Q1.4', 'Q2.1', 'Q2.2', 'Q2.3', 'Q3.2_1', 'Q3.2_2', 'Q3.2_3', 'Q3.3_1', 'Q3.3_2', 'Q3.3_3', 'Q3.4_1', 'Q3.4_2', 'Q3.4_3', 'Q3.5_1', 'Q3.5_2', 'Q3.5_3', 'Q3.5_4', 'Q3.6_1', 'Q3.6_2', 'Q3.6_3', 'Q3.6_4', 'Q3.7_1', 'Q3.7_2', 'Q3.7_3', 'Q3.8_1', 'Q3.8_2', 'Q3.8_3', 'Q3.9', 'Q3.10_1', 'Q3.10_2', 'Q3.10_3', 'Q3.10_4', 'Q3.10_5', 'Q3.12', 'Q3.12_4_TEXT', 'Q3.13', 'prior legal experien', 'Q3.14', 'Q3.15', 'Q3.15_2_TEXT', 'Q3.16', 'Q3.16_2_TEXT', 'Q3.17', 'Q3.18', 'Admin1_1', 'Admin1_2', 'Admin1_3', 'Admin1_36', 'Admin1_37', 'Admin1_38', 'chatbotRightTrack', 'doneManually', 'experimentId', 'group', 'Start Time', 'End Time']


In [14]:
# Group A = EmpathyBot, Anger
# Group B = EmpathyBot, NoAnger
# Group C = NoEmpathyBot, Anger
# Groud D = NoEmpathyBot, NoAnger
# Group E = FAQ, Anger
# Group F = FAQ, NoAnger

# Original Coding from Firebase

In [3]:
# Groups

data_all['EmpathyBot'] = data_all['group'].copy().map({'A':'Empathy', 'B':'Empathy', 'C': 'NoEmpathy', 'D': 'NoEmpathy', 'E': 'FAQ', 'F': 'FAQ'})
data_all['AngerInducement'] = data_all['group'].copy().map({'A':'NoAnger', 'C':'NoAnger', 'E':'NoAnger', 'B':'Anger', 'D':'Anger', 'F':'Anger'})
group = data_all[['group', 'EmpathyBot', 'AngerInducement']]
group.to_csv("data/group.csv",header='group',index=False)

In [4]:
# Demographics

demographics_keys = ['Q1','Q2','Q3_1','Q1_3','Q42','group']
demo_key_values = ['adult', 'student', 'zipcode', 'age','prior_experience','group']
data = data_all[demographics_keys]
data.to_csv('data/demographics.csv', header = demo_key_values, index=False)


In [ ]:


detailed_demographics_keys = ['Q3.12','Q3.12_4_TEXT','Q3.13','prior legal experien','Q3.14','Q3.15','Q3.15_2_TEXT','Q3.16','Q3.16_2_TEXT','Q3.17','Q3.18']
detailed_demo_key_values = ['Gender','Gender_Text','Degree','Law_Ex','Race','P_Language','P_Lan_Text','S_Language','S_Lan_Text','Work','Income']
data = data_all[detailed_demographics_keys]
data.to_csv('data/detailed_demographics.csv', header = detailed_demo_key_values, index=False)

In [5]:
# Comprehension Questions
comprehension_q_keys = ['Q1.2','Q1.3','Q1.4','Q2.1','Q2.2','Q2.3']
data = data_all[comprehension_q_keys]
data.to_csv('data/comprehensions.csv',header=comprehension_q_keys, index=False)

In [6]:
comprehensions = pd.read_csv("data/comprehensions.csv")

comprehensions['Q1_2'] = comprehensions['Q1.2'].map(lambda x: x == 'May be able to deduct past-due rent from your security deposit')
comprehensions['Q1_3'] = comprehensions['Q1.3'].map(lambda x: x == '45 days')
comprehensions['Q1_4'] = comprehensions['Q1.4'].map(lambda x: x == '30 days')
comprehensions['Q2_1'] = comprehensions['Q2.1'].map(lambda x: x == '$2000')
comprehensions['Q2_2'] = comprehensions['Q2.2'].map(lambda x: x == 'Send a letter to the landlord demanding the return of the security deposit')
comprehensions['Q2_3'] = comprehensions['Q2.3'].map(lambda x: x == 'The landlord failed to send receipts or an estimate to the tenant within the required time')

In [7]:
comprehensions['Q1_2'] = comprehensions['Q1_2'].replace({True: 1, False: 0})
comprehensions['Q1_3'] = comprehensions['Q1_3'].replace({True: 1, False: 0})
comprehensions['Q1_4'] = comprehensions['Q1_4'].replace({True: 1, False: 0})
comprehensions['Q2_1'] = comprehensions['Q2_1'].replace({True: 1, False: 0})
comprehensions['Q2_2'] = comprehensions['Q2_2'].replace({True: 1, False: 0})
comprehensions['Q2_3'] = comprehensions['Q2_3'].replace({True: 1, False: 0})

In [8]:
comp = comprehensions[['Q1_2', 'Q1_3', 'Q1_4', 'Q2_1', 'Q2_2', 'Q2_3']].to_numpy()
counts = np.count_nonzero(comp, axis=1)
comprehensions['ComprehensionCount'] = counts

In [9]:
comprehensions.to_csv('data/comprehensions.csv', index=False)

In [11]:
# Dependent Variables

# ## Helpfulness
keys = ['Q3.2_1','Q3.2_2','Q3.2_3']
values = ['helpfulness','usefulness','informative']
data = data_all[keys]
data.to_csv("data/dependent_variables/helpfulness.csv",header=values,index=False)

means = data_all.groupby(['group'])[keys].mean().round(2)
stds = data_all.groupby(['group'])[keys].std().round(2)
print(means)
print(stds)

# ## Psychological Safety
# keys = ['Q3.4_1','Q3.4_2','Q3.4_3']
# values = ['CannotTrust','FearCantSatisfy','FrustratedDissatisfied']
mapping = {'Agree':7, 'Strongly Agree':6, 'Somewhat Agree':5, 'Neutral':4, 'Somewhat Disagree':3, 'Strongly Disagree':2, 'Disagree':1}
# data = data_all[keys]
# ndata = data.applymap(lambda x: mapping.get(x))
# ndata.to_csv("data/dependent_variables/psychological_safety.csv",header=values,index=False)

## Credibility
keys = ['Q3.5_1','Q3.5_2','Q3.5_3','Q3.5_4']
values = ['Trustworthy','Honesty','Ability','OverallTrust']
# print(data_all[keys].head)
data_all.replace(mapping, inplace=True)
data = data_all[keys]
# Mapping is the same
data.to_csv("data/dependent_variables/credibility.csv",header=values,index=False)

# print(data_all[keys].head)
# print(data_all.head)
# print(data_all.columns)

means = data_all.groupby(['group'])[keys].mean().round(2)
stds = data_all.groupby(['group'])[keys].std().round(2)
print(means)
print(stds)

## Cognitive Effort
keys = ['Q3.6_1','Q3.6_2','Q3.6_3','Q3.6_4']
values = ['Thinking','Contemplated','Demanding','Effort']
data = data_all[keys]
# Mapping is the same
data.to_csv("data/dependent_variables/cognitive_effort.csv",header=values,index=False)

means = data_all.groupby(['group'])[keys].mean().round(2)
stds = data_all.groupby(['group'])[keys].std().round(2)
print(means)
print(stds)



       Q3.2_1  Q3.2_2  Q3.2_3
group                        
A        6.53    6.53    6.34
B        6.40    6.45    6.30
C        6.13    6.20    5.93
D        5.96    6.11    6.20
E        6.11    6.09    6.17
F        6.00    6.07    6.28
       Q3.2_1  Q3.2_2  Q3.2_3
group                        
A        0.75    0.80    0.98
B        1.01    0.95    0.98
C        1.34    1.38    1.45
D        1.45    1.30    1.25
E        1.10    1.15    1.08
F        0.94    1.08    0.93
       Q3.5_1  Q3.5_2  Q3.5_3  Q3.5_4
group                                
A        6.17    6.11    5.51    6.00
B        5.47    5.23    4.55    5.30
C        5.70    5.63    4.35    5.33
D        5.67    5.78    4.40    5.36
E        6.00    5.57    4.57    5.78
F        6.20    5.50    5.04    5.85
       Q3.5_1  Q3.5_2  Q3.5_3  Q3.5_4
group                                
A        1.15    1.32    1.68    1.23
B        1.65    1.90    1.87    1.65
C        1.23    1.25    1.86    1.27
D        1.15    1.41    1

In [12]:
# Moderating Varaibles

## Anger
keys = ['Q3.3_1','Q3.3_2','Q3.3_3']
values = ['Mad','Angry','Furious']
data = data_all[keys]
# Mapping is the same
ndata = data.applymap(lambda x: mapping.get(x))
ndata.to_csv("data/moderating_variables/anger.csv",header=values,index=False)

## Online Preference
keys = ['Q3.8_1','Q3.8_2','Q3.8_3']
values = ['Perfer','Comfort','online']
data = data_all[keys]
# Mapping is the same
ndata = data.applymap(lambda x: mapping.get(x))
ndata.to_csv("data/moderating_variables/online_preference.csv",header=values,index=False)

# Need for Empathy
# keys = ['Q3.10_1','Q3.10_2','Q3.10_3','Q3.10_4','Q3.10_5']
# values = ['Compassionate','Realize','Misfortune','TenderFeelings','Friend']
## Removed Realize
keys = ['Q3.10_1','Q3.10_3','Q3.10_4','Q3.10_5']
values = ['Compassionate','Misfortune','TenderFeelings','Friend']
## Removed Friend
# keys = ['Q3.10_1','Q3.10_2','Q3.10_3','Q3.10_4']
# values = ['Compassionate','Realize','Misfortune','TenderFeelings']
data = data_all[keys]
# Mapping is the same
ndata = data.applymap(lambda x: mapping.get(x))
ndata.to_csv("data/moderating_variables/empathy.csv",header=values,index=False)

## Chatbot Familiarity
key = ['Q3.9']
value = ['Familiarity']
data = data_all[key]
mapping = {'No':0, 'Yes':1}
ndata = data.applymap(lambda x: mapping.get(x))
ndata.to_csv("data/moderating_variables/chatbot_familiarity.csv",header=value,index=False)

In [13]:
# Get the mean and standard deviation of each dependent variable


In [300]:
# Cronbach's alpha

# dependent_variables
l = loader()
dependent_variables, moderating_variables = l.load_data()

dependent_variables_alphas = {}
for data_name in dependent_variables.keys():
    dependent_variables_alphas[data_name] = pg.cronbach_alpha(data=dependent_variables.get(data_name))
print(dependent_variables_alphas)

# Moderating Variables
moderating_variables_alphas = {}
for data_name in moderating_variables.keys():
    if data_name != 'chatbot_familiarity':
    # chatbot_familiarity cannot do cronbach's alpha because it only have one item
        moderating_variables_alphas[data_name] = pg.cronbach_alpha(data=moderating_variables[data_name])
print(moderating_variables_alphas)

{'credibility': (0.8314952217091008, array([0.797, 0.862])), 'cognitive_effort': (0.8678723211232615, array([0.841, 0.892])), 'helpfulness': (0.9299553662000406, array([0.914, 0.943]))}
{'anger': (nan, array([nan, nan])), 'online_preference': (nan, array([nan, nan])), 'empathy': (nan, array([nan, nan]))}
